# Pajala ARV Flöden
Data leverarad av Kristofer Grammer <kristofer.gramner@gefasystem.se>.

In [17]:
# .\.venv\Scripts\Activate.ps1

In [18]:
import inspect
import re
from pathlib import Path
import pandas as pd
def dprint(x): # https://stackoverflow.com/questions/32000934/print-a-variables-name-and-value/57225950#57225950
    frame = inspect.currentframe().f_back
    s = inspect.getframeinfo(frame).code_context[0]
    r = re.search(r"\((.*)\)", s).group(1)
    print("{} = {}".format(r,x))

def _pk1_path_for_xlsx(xlsx_path):
    """Return Path for .pk1 cache file stored next to the xlsx with same base name and extension '.pk1'."""
    p = Path(xlsx_path) if not isinstance(xlsx_path, Path) else xlsx_path
    return p.with_suffix('.pk1')

def load_or_cache_excel(xlsx_path, read_kwargs=None, force_refresh=False):
    """Load DataFrame from a .pk1 cache next to the xlsx if present; otherwise read the xlsx and save the .pk1.
    Returns the DataFrame.
    read_kwargs: dict forwarded to pd.read_excel.
    force_refresh: if True, re-read the Excel and overwrite cache."""
    read_kwargs = read_kwargs or {}
    pk1 = _pk1_path_for_xlsx(xlsx_path)
    if pk1.exists() and not force_refresh:
        try:
            df = pd.read_pickle(pk1)
            print(f'Loaded cache {pk1}')
            return df
        except Exception as e:
            print(f'Warning: failed to load {pk1} (will re-read Excel): {e}')
    # read Excel and attempt to save cache
    df = pd.read_excel(xlsx_path, **read_kwargs)
    try:
        df.to_pickle(pk1)
        print(f'Saved cache {pk1}')
    except Exception as e:
        print(f'Warning: could not save cache {pk1}: {e}')
    return df

In [19]:
# Read Excel files (with pk1 cache next to each xlsx). Uses load_or_cache_excel from cell 2.
excel_file_path_FT_10101 = r'C:\Users\chrini\OneDrive - Norconsult Group\Projekt\1097224_Pajala ARV\4 Underlag\04 Underkonsult, Sidokonsult\gefasystem.app.box.com\Flode_Pajala_RV\FT-10101.xlsx'
excel_file_path_FT_72101 = r'c:\Users\chrini\OneDrive - Norconsult Group\Projekt\1097224_Pajala ARV\4 Underlag\04 Underkonsult, Sidokonsult\gefasystem.app.box.com\Flode_Pajala_RV\FT-72101.xlsx'
# Load using the helper which places the .pk1 next to the xlsx with same base name
df_1 = load_or_cache_excel(excel_file_path_FT_10101)
df_2 = load_or_cache_excel(excel_file_path_FT_72101)

Loaded cache C:\Users\chrini\OneDrive - Norconsult Group\Projekt\1097224_Pajala ARV\4 Underlag\04 Underkonsult, Sidokonsult\gefasystem.app.box.com\Flode_Pajala_RV\FT-10101.pk1
Loaded cache c:\Users\chrini\OneDrive - Norconsult Group\Projekt\1097224_Pajala ARV\4 Underlag\04 Underkonsult, Sidokonsult\gefasystem.app.box.com\Flode_Pajala_RV\FT-72101.pk1


In [20]:
# dprint(df_1.head())
df_1.rename(columns={'TimeDate': 'DateTime', 'Val':'Inflöde FT-10101'}, inplace=True)
df_1['DateTime'] = pd.to_datetime(df_1['DateTime'])
df_1.drop(columns=['ID', 'TimeLength'], inplace=True)
df_1.set_index('DateTime', inplace=True)
# dprint(df_1.head())


# print(df_2.head())
df_2.rename(columns={'TimeDate': 'DateTime', 'Val':'Utflöde FT-72101'}, inplace=True)
df_2['DateTime'] = pd.to_datetime(df_2['DateTime'])
df_2.drop(columns=['ID', 'TimeLength'], inplace=True)
df_2.set_index('DateTime', inplace=True)
dprint(df_2.head())

# Merge the two DataFrames on the DateTime index, aligning values
# Using merge instead of concat to handle any duplicate indices
df_ax = pd.merge(df_1, df_2, left_index=True, right_index=True, how='outer')

# Show the result
print("\nMerged DataFrame:")
dprint(df_ax.head())

# Check for any missing values after merge
print("\nMissing values in merged DataFrame:")
dprint(df_ax.isna().sum())

df_2.head() =                      Utflöde FT-72101
DateTime                             
2024-11-01 00:00:00         26.369708
2024-11-01 00:01:00         75.286610
2024-11-01 00:02:00         68.255072
2024-11-01 00:03:00         26.485958
2024-11-01 00:04:00         15.062156

Merged DataFrame:
df_ax.head() =                      Inflöde FT-10101  Utflöde FT-72101
DateTime                                               
2024-11-01 00:00:00         50.144617         26.369708
2024-11-01 00:01:00         48.459584         75.286610
2024-11-01 00:02:00         51.117986         68.255072
2024-11-01 00:03:00         23.234605         26.485958
2024-11-01 00:04:00         76.512842         15.062156

Missing values in merged DataFrame:
df_ax.isna().sum() = Inflöde FT-10101     0
Utflöde FT-72101    10
dtype: int64


In [21]:
# Show all column names to verify what needs renaming
print("Columns in df_1:", df_1.columns.tolist())
print("Columns in df_2:", df_2.columns.tolist())
print("Columns in df_2:", df_ax.columns.tolist())

Columns in df_1: ['Inflöde FT-10101']
Columns in df_2: ['Utflöde FT-72101']
Columns in df_2: ['Inflöde FT-10101', 'Utflöde FT-72101']


In [22]:
# Show DateTime indices where there are missing values
print("\nDateTime indices with missing FT-10101:")
print(df_ax[df_ax['Inflöde FT-10101'].isna()].index.strftime('%Y-%m-%d %H:%M:%S').tolist())

print("\nDateTime indices with missing FT-72101:")
print(df_ax[df_ax['Utflöde FT-72101'].isna()].index.strftime('%Y-%m-%d %H:%M:%S').tolist())

# Print summary of gaps
print("\nSummary of gaps:")
print(f"Total rows in merged DataFrame: {len(df_ax)}")
print(f"Rows with missing FT-10101: {df_ax['Inflöde FT-10101'].isna().sum()}")
print(f"Rows with missing FT-72101: {df_ax['Utflöde FT-72101'].isna().sum()}")
print(f"Rows with data in both columns: {len(df_ax) - df_ax.isna().any(axis=1).sum()}")


DateTime indices with missing FT-10101:
[]

DateTime indices with missing FT-72101:
['2025-10-30 10:26:00', '2025-10-30 10:27:00', '2025-10-30 10:28:00', '2025-10-30 10:29:00', '2025-10-30 10:30:00', '2025-10-30 10:31:00', '2025-10-30 10:32:00', '2025-10-30 10:33:00', '2025-10-30 10:34:00', '2025-10-30 10:35:00']

Summary of gaps:
Total rows in merged DataFrame: 506547
Rows with missing FT-10101: 0
Rows with missing FT-72101: 10
Rows with data in both columns: 506537


# Calculate Moving Averages
Let's calculate moving averages with different window sizes to smooth the time series data.

In [23]:
# Calculate moving averages for each column (time-based windows)
# Each row in df_ax represents 1 minute, so use time-based rolling windows
import pandas as pd

# Prepare container for moving averages
df_ma = pd.DataFrame(index=df_ax.index)

# Define time-based windows (labels -> pandas offset strings)
windows = { '1h': '60min', '24h': '24H', '7d': '7D' }

for col in df_ax.columns:
    for w_label, w_offset in windows.items():
        # Use time-based rolling which is robust to missing/irregular timestamps
        ma = df_ax[col].rolling(w_offset, min_periods=1).mean()
        ma_col_name = f"{col}_MA_{w_label}"
        df_ma[ma_col_name] = ma

# Compute differences (Inflöde - Utflöde) for each moving-average window in a separate DataFrame
# Source column names in df_ax are 'Inflöde FT-10101' and 'Utflöde FT-72101'
df_ma_diff = pd.DataFrame(index=df_ax.index)
left_base = 'Inflöde FT-10101'
right_base = 'Utflöde FT-72101'
for w_label in windows.keys():
    left_col = f"{left_base}_MA_{w_label}"
    right_col = f"{right_base}_MA_{w_label}"
    diff_col = f"Diff_MA_{w_label}"
    if left_col in df_ma.columns and right_col in df_ma.columns:
        df_ma_diff[diff_col] = (df_ma[left_col] - df_ma[right_col])/df_ma[left_col]
    else:
        # If one of the MA columns is missing, create the diff column with NaNs and warn
        df_ma_diff[diff_col] = pd.NA
        print(f"Warning: cannot compute {diff_col} because {left_col} or {right_col} is missing")

# C side-by-side
# concat keeps the original index and places MA & diff columns next to original data

df_ax_ma = pd.concat([df_ax, df_ma], axis=1)
df_ax_ma_diff = pd.concat([df_ax, df_ma, df_ma_diff], axis=1)

dprint(len(df_ax_ma_diff.columns))

# Display first few rows of moving averages
print("\nFirst few rows of moving averages:")
print(df_ax_ma_diff.head())

# Display first few rows of differences
print("\nFirst few rows of differences:")
print(df_ax_ma_diff.head())

# Display basic statistics of original vs smoothed data
print("\nComparison of statistics between original and moving averages (original data stats):")
print(df_ax_ma_diff.describe())

len(df_ax_ma_diff.columns) = 11

First few rows of moving averages:
                     Inflöde FT-10101  Utflöde FT-72101  \
DateTime                                                  
2024-11-01 00:00:00         50.144617         26.369708   
2024-11-01 00:01:00         48.459584         75.286610   
2024-11-01 00:02:00         51.117986         68.255072   
2024-11-01 00:03:00         23.234605         26.485958   
2024-11-01 00:04:00         76.512842         15.062156   

                     Inflöde FT-10101_MA_1h  Inflöde FT-10101_MA_24h  \
DateTime                                                               
2024-11-01 00:00:00               50.144617                50.144617   
2024-11-01 00:01:00               49.302101                49.302101   
2024-11-01 00:02:00               49.907396                49.907396   
2024-11-01 00:03:00               43.239198                43.239198   
2024-11-01 00:04:00               49.893927                49.893927   

             

C:\Users\chrini\AppData\Local\Temp\ipykernel_55340\1676729539.py:14: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  ma = df_ax[col].rolling(w_offset, min_periods=1).mean()
C:\Users\chrini\AppData\Local\Temp\ipykernel_55340\1676729539.py:14: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  ma = df_ax[col].rolling(w_offset, min_periods=1).mean()


       Inflöde FT-10101  Utflöde FT-72101  Inflöde FT-10101_MA_1h  \
count     506547.000000     506537.000000           506547.000000   
mean          35.128978         32.611549               35.130383   
std           13.524475         17.969017               11.303725   
min           -0.000015          0.000000                0.000000   
25%           27.353065         17.380294               27.811552   
50%           33.964639         33.586557               34.650663   
75%           40.936627         45.662986               41.657167   
max          188.413661        125.125472              166.498410   

       Inflöde FT-10101_MA_24h  Inflöde FT-10101_MA_7d  \
count            506547.000000           506547.000000   
mean                 35.143075               35.156271   
std                   8.812788                7.877522   
min                  21.403293               22.364565   
25%                  28.158768               28.317741   
50%                  34.679362

In [ ]:
from PyQt6.QtWidgets import QApplication, QMainWindow, QVBoxLayout, QHBoxLayout, QCheckBox, QPushButton, QWidget, QLabel
from PyQt6.QtGui import QKeySequence, QShortcut
from matplotlib.figure import Figure
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas, NavigationToolbar2QT
import matplotlib.dates as mdates
from matplotlib.dates import MO, WeekdayLocator
import matplotlib.pyplot as plt
import sys
import numpy as np
import pandas as pd

import inspect
import re
def dprint(x): # https://stackoverflow.com/questions/32000934/print-a-variables-name-and-value/57225950#57225950
    frame = inspect.currentframe().f_back
    s = inspect.getframeinfo(frame).code_context[0]
    r = re.search(r"\((.*)\)", s).group(1)
    print("{} = {}".format(r,x))

class CustomNavigationToolbar(NavigationToolbar2QT):
    def __init__(self, canvas, parent=None, plot_window = None):
        super().__init__(canvas, parent)
        self.plot_window = plot_window
        self.addSeparator()
        self.addAction('Autoscale', self.autoscale)
        self.addSeparator()
        self.addAction('Autoscale LY', self.autoscaleLeftY)
        self.addSeparator()
        self.addAction('Autoscale RY', self.autoscaleRightY)
        self.addSeparator()        
        self.addAction('Move Left', self.moveLeft)
        self.addSeparator()
        self.addAction('Move Right', self.moveRight)
        self.addSeparator()
        self.addAction('Reset X', self.plot_window.reformatXAxis)
        self.addSeparator()
#         # Set fixed size for buttons
#         button_width = 100
#         button_height = 30
        # Add keyboard shortcuts
        self.shortcut_pan = QShortcut(QKeySequence("P"), self)
        self.shortcut_pan.activated.connect(self.pan)

        self.shortcut_zoom = QShortcut(QKeySequence("Z"), self)
        self.shortcut_zoom.activated.connect(self.zoom)

        # Add QLabel to display xAxisLen
        self.xAxisLenLabel = QLabel("xAxisLen: N/A")
        self.addWidget(self.xAxisLenLabel)

    def autoscale(self):
        ax1 = self.plot_window.axL
        ax1.autoscale(axis='both')
        self.canvas.draw()
        self.update_xAxisLen()

    def autoscaleLeftY(self):
        df_axL = self.plot_window.df_axL
        series_visibleLeftY = self.plot_window.series_visible[:len(df_axL.columns)]
        if not any(series_visibleLeftY):
            return
        # Get current x-axis limits
        xlim = self.plot_window.axL.get_xlim()
        xlim = [mdates.num2date(x).replace(tzinfo=None) for x in xlim]
        selected_columns = df_axL[(df_axL.index >= xlim[0]) & (df_axL.index <= xlim[1])]
        selected_columns = selected_columns.loc[:, series_visibleLeftY]
        if selected_columns.empty:
            return
        ylim = (selected_columns.min().min(), selected_columns.max().max())
        self.plot_window.axL.set_ylim(ylim)
        self.canvas.draw()

    def autoscaleRightY(self):
        # Guard if there is no right-axis data
        df_axR = self.plot_window.df_axR
        if df_axR is None or df_axR.empty:
            return
        # compute visible columns for right axis
        df_axL = self.plot_window.df_axL
        total_left = len(df_axL.columns)
        series_visibleRightY = self.plot_window.series_visible[total_left: total_left + len(df_axR.columns)]
        if not any(series_visibleRightY):
            return
        # Get current x-axis limits
        xlim = self.plot_window.axR.get_xlim()
        xlim = [mdates.num2date(x).replace(tzinfo=None) for x in xlim]
        selected_columns = df_axR[(df_axR.index >= xlim[0]) & (df_axR.index <= xlim[1])]
        selected_columns = selected_columns.loc[:, series_visibleRightY]
        if selected_columns.empty:
            return
        ylim = (selected_columns.min().min(), selected_columns.max().max())
        self.plot_window.axR.set_ylim(ylim)
        self.canvas.draw()

    def pan(self):
        super().pan()
        self.plot_window.reformatXAxis()
        self.update_xAxisLen()

    def zoom(self):
        super().zoom()
        self.plot_window.reformatXAxis()
        self.update_xAxisLen()

    def update_xAxisLen(self):
        ax1 = self.canvas.figure.gca()
        xlim = ax1.get_xlim()
        xAxisLen = xlim[1] - xlim[0]
        self.xAxisLenLabel.setText(f"xAxisLen: {xAxisLen:.2f}")
    
    def moveLeft(self):
        xlim = self.plot_window.axL.get_xlim()
        step = (xlim[1] - xlim[0]) * 0.25  # Move 25% of the current x-axis range
        self.plot_window.axL.set_xlim(xlim[0] - step, xlim[1] - step)
        self.canvas.draw()
        self.update_xAxisLen()

    def moveRight(self):
        xlim = self.plot_window.axL.get_xlim()
        step = (xlim[1] - xlim[0]) * 0.25  # Move 25% of the current x-axis range
        self.plot_window.axL.set_xlim(xlim[0] + step, xlim[1] + step)
        self.canvas.draw()
        self.update_xAxisLen()

class InteractivePlotWindow(QMainWindow):
    def __init__(self, df_axL, df_axL_Title = None, df_axR=None, df_axR_Title = None, WindowTitle = None, initial_visible=None):
        super().__init__()
        if WindowTitle is None:
            self.setWindowTitle("Interactive Plot")
        else:
            self.setWindowTitle(WindowTitle)

        # Store the DataFrames (ensure df_axR is a DataFrame if None)
        self.df_axL = df_axL
        self.df_axR = df_axR if df_axR is not None else pd.DataFrame()
        self.df_axL_Title = df_axL_Title
        self.df_axR_Title = df_axR_Title
        
        # Flag to check if it's the initial plot
        self.initial_plot = True

        # Create central widget
        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)
        layout = QVBoxLayout(self.central_widget)

        # Create Figure and Canvas
        # Use constrained_layout and tighter subplot margins to reduce border whitespace
        self.fig = Figure(constrained_layout=True)
        self.canvas = FigureCanvas(self.fig)
        self.toolbar = CustomNavigationToolbar(self.canvas, self, plot_window=self)

        layout.addWidget(self.toolbar)
        layout.addWidget(self.canvas)

        # Determine columns and initial visibility
        left_cols = list(self.df_axL.columns)
        right_cols = list(self.df_axR.columns) if (self.df_axR is not None and not self.df_axR.empty) else []
        all_cols = left_cols + right_cols

        # initial_visible may be:
        # - None (all True),
        # - list of 1/0 or booleans of same length as all_cols,
        # - list of column-names to show.
        if initial_visible is None:
            self.series_visible = [True] * len(all_cols)
        elif isinstance(initial_visible, (list, tuple)) and len(initial_visible) == len(all_cols):
            # Convert numeric 1/0 or booleans to booleans
            self.series_visible = [bool(x) for x in initial_visible]
        else:
            # Treat initial_visible as list of column names to show
            try:
                visible_set = set(initial_visible)
                self.series_visible = [ (col in visible_set) for col in all_cols ]
            except Exception:
                # fallback to all True
                self.series_visible = [True] * len(all_cols)

        # Build checkboxes in the same column order (left then right)
        checkbox_layout = QHBoxLayout()
        self.checkboxes = []
        for i, col in enumerate(all_cols):
            checkbox = QCheckBox(f"{col}")
            checkbox.setChecked(bool(self.series_visible[i]))
            checkbox.stateChanged.connect(self.create_toggle_function(i))
            checkbox_layout.addWidget(checkbox)
            self.checkboxes.append(checkbox)

        layout.addLayout(checkbox_layout)

        # Initial plot
        self.plot()

    def create_toggle_function(self, index):
        def toggle():
            self.series_visible[index] = not self.series_visible[index]
            self.plot()
        return toggle

    def plot(self):
        # Save the axis title and labels
        title = self.axL.get_title() if hasattr(self, 'axL') else ''
        xlabel = self.axL.get_xlabel() if hasattr(self, 'axL') else ''
        ylabel = self.axL.get_ylabel() if hasattr(self, 'axL') else ''

        if not self.initial_plot:
            try:
                self.axLxlim = self.axL.get_xlim()
                self.axLylim = self.axL.get_ylim()
                if self.df_axR is not None and not self.df_axR.empty and self.axR is not None:
                    self.axRxlim = self.axR.get_xlim()
                    self.axRylim = self.axR.get_ylim()
            except Exception:
                pass

        # Clear the axes
        self.fig.clear()
        self.axL = self.fig.add_subplot(111)

        # Restore the axis title and labels
        self.axL.set_title(title)
        self.axL.set_xlabel(xlabel)
        self.axL.set_ylabel(ylabel)

        self.axL.grid(visible=True, which='major', axis='both', color='grey')
        self.axL.grid(visible=True, which='minor', axis='both', color='lightgrey')
        self.axL.tick_params(which='minor', labelcolor='lightgrey')
        self.axL.tick_params(axis='x', rotation=90, which='both')
        if self.df_axL_Title:
            self.axL.set_ylabel(self.df_axL_Title)

        # Plot left-axis columns
        for i, col in enumerate(self.df_axL.columns):
            if self.series_visible[i]:
                self.axL.plot(self.df_axL.index, self.df_axL[col], label=col, alpha=0.5)

        handles, labels = self.axL.get_legend_handles_labels()

        # Reset right axis
        self.axR = None
        if self.df_axR is not None and not self.df_axR.empty:
            self.axR = self.axL.twinx()
            for i, col in enumerate(self.df_axR.columns, start=len(self.df_axL.columns)):
                if self.series_visible[i]:
                    # align on left x index (assumes same index or compatible)
                    self.axR.plot(self.df_axL.index, self.df_axR[col], label=col, alpha=0.5)
            if self.df_axR_Title:
                self.axR.set_ylabel(self.df_axR_Title)

            handles2, labels2 = self.axR.get_legend_handles_labels()
            handles += handles2
            labels += labels2

        self.axL.legend(handles, labels)

        if self.initial_plot:
            self.initial_plot = False
            self.axL.autoscale(axis='both')
            try:
                self.axLxlim = self.axL.get_xlim()
                self.axLylim = self.axL.get_ylim()
            except Exception:
                pass
            if self.df_axR is not None and not self.df_axR.empty and self.axR is not None:
                try:
                    self.axRxlim = self.axR.get_xlim()
                    self.axRylim = self.axR.get_ylim()
                except Exception:
                    pass
            # Apply a tighter layout/margins so the figure uses more canvas space
            # constrained_layout active; manual subplots_adjust removed to avoid conflict
        else:
            try:
                self.axL.set_xlim(self.axLxlim)
                self.axL.set_ylim(self.axLylim)
                if self.df_axR is not None and not self.df_axR.empty and self.axR is not None:
                    self.axR.set_xlim(self.axRxlim)
                    self.axR.set_ylim(self.axRylim)
            except Exception:
                pass

        # Apply x-axis formatting to both axL and axR
        self.reformatXAxis()

        # constrained_layout active; manual subplots_adjust removed to avoid conflict

        self.canvas.draw()

    def apply_xaxis_formatting(self, ax):
        xlim = ax.get_xlim()
        xAxisLen = xlim[1] - xlim[0]

        if xAxisLen < 3 / 24:
            ax.xaxis.set_major_locator(mdates.HourLocator(byhour=range(24), interval=1))
            ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=range(60), interval=1))
            ax.xaxis.set_major_formatter(mdates.DateFormatter(r'%Y-%m-%d %H:%M'))
            ax.xaxis.set_minor_formatter(mdates.DateFormatter(r'%H:%M'))
        elif xAxisLen < 6 / 24:
            ax.xaxis.set_major_locator(mdates.HourLocator(byhour=range(24), interval=1))
            ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=range(60), interval=15))
            ax.xaxis.set_major_formatter(mdates.DateFormatter(r'%Y-%m-%d %H:%M'))
            ax.xaxis.set_minor_formatter(mdates.DateFormatter(r'%H:%M'))
        elif xAxisLen < 2:
            ax.xaxis.set_major_locator(mdates.DayLocator(bymonthday=range(1, 32), interval=1))
            ax.xaxis.set_minor_locator(mdates.HourLocator(byhour=range(24), interval=1))
            ax.xaxis.set_major_formatter(mdates.DateFormatter(r'%Y-%m-%d %H:%M'))
            ax.xaxis.set_minor_formatter(mdates.DateFormatter(r'%H:%M'))
        elif xAxisLen < 7:
            ax.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=MO, interval=1))
            ax.xaxis.set_minor_locator(mdates.HourLocator(interval=1))
            ax.xaxis.set_major_formatter(mdates.DateFormatter(r'W%U-%m-%d %H:%M'))
            ax.xaxis.set_minor_formatter(mdates.DateFormatter(r'%Y-%m-%d %H:%M'))
        elif xAxisLen < 60:
            ax.xaxis.set_major_locator(mdates.MonthLocator(bymonthday=1, interval=1))
            ax.xaxis.set_minor_locator(mdates.WeekdayLocator(byweekday=MO, interval=1))
            ax.xaxis.set_major_formatter(mdates.DateFormatter(r'%Y-%m-%d'))
            ax.xaxis.set_minor_formatter(mdates.DateFormatter(r'W%U-%m-%d'))
        elif xAxisLen < 367*1.5:
            ax.xaxis.set_major_locator(mdates.YearLocator(base=1,month=1))
            ax.xaxis.set_minor_locator(mdates.MonthLocator(bymonthday=1, interval=1))
            ax.xaxis.set_major_formatter(mdates.DateFormatter(r'%Y'))
            ax.xaxis.set_minor_formatter(mdates.DateFormatter(r'%Y-%m-%d'))
        else:
            ax.xaxis.set_major_locator(mdates.YearLocator(base=10,month=1))
            ax.xaxis.set_minor_locator(mdates.YearLocator(base=1,month=1))
            ax.xaxis.set_major_formatter(mdates.DateFormatter(r'%Y'))
            ax.xaxis.set_minor_formatter(mdates.DateFormatter(r'%Y'))
        return ax

    def reformatXAxis(self):
        self.axL = self.apply_xaxis_formatting(self.axL)
        if self.axR:
            self.axR = self.apply_xaxis_formatting(self.axR)

if __name__ == "__main__":
    import sys as _sys
    from IPython import get_ipython

    def _make_and_show():
        app = QApplication.instance() or QApplication(_sys.argv)
        mainWin = InteractivePlotWindow(df_axL = df_ax_ma,
                            df_axL_Title = 'Flöde [m3/h]', 
                            df_axR = df_ma_diff, 
                            df_axR_Title = 'Flödesdiff [(in-ut)/in]',
                            WindowTitle='Pajala ARV Flöde',
                            initial_visible = [
                                0,0,  # Original columns
                                0,0,  # 1h MA
                                0,0,  # 24h MA
                                0,0,  # 7d MA
                                1,  # Diff_MA_1h
                                1,  # Diff_MA_24h
                                1   # Diff_MA_7d
                            ]
                        )
        mainWin.show()
        # Keep references to avoid garbage collection in notebook kernels.
        # Store on the app and module globals so the objects persist after this function returns.
        try:
            app._pajala_mainWin = mainWin
        except Exception:
            pass
        globals()['_pajala_mainWin'] = mainWin
        globals()['_pajala_app'] = app
        return app

    # If running inside an IPython kernel (notebook), request IPython to enable the Qt event loop
    if 'ipykernel' in _sys.modules:
        try:
            ip = get_ipython()
            if ip is not None:
                # enable GUI event loop integration; this avoids a blocking app.exec() call
                ip.run_line_magic('gui', 'qt')
        except Exception:
            ip = None
        # Create and show window but do NOT call app.exec() - the event loop is managed by IPython
        app = _make_and_show()
        # Keep references in the IPython user namespace if available so users can interact with them
        if ip is not None:
            try:
                ip.user_ns['_pajala_app'] = app
                ip.user_ns['_pajala_mainWin'] = globals().get('_pajala_mainWin')
            except Exception:
                # Fall back to module globals (already set by _make_and_show)
                pass
    else:
        # Running as a script: start the blocking event loop
        app = _make_and_show()
        _sys.exit(app.exec())